In [32]:
import json
with open('/tmp/food.jsonl') as f:
    for line in f:
        j_content = json.loads(line)
        print(j_content)

['Coffee order: cappuccino', {'entities': [[0, 6, 'food'], [14, 24, 'food']]}]
["Cholula's Hot Sauce is her favorite.", {'entities': [[0, 19, 'food']]}]
['fav ice cream: mint chocolate chip', {'entities': [[4, 13, 'food'], [15, 34, 'food']]}]
['fav drink: lemonade', {'entities': [[11, 19, 'food']]}]


In [43]:
TRAIN_DATA=[
    #['Coffee order: cappuccino', {'entities': [[0, 6, 'food'], [14, 24, 'food']]}],
           ["Cholula's Hot Sauce is her favorite.", {'entities': [[0, 19, 'food']]}],
          # ['fav ice cream: mint chocolate chip', {'entities': [[4, 13, 'food'], [15, 34, 'food']]}],
           ['fav drink: lemonade', {'entities': [[11, 19, 'food']]}],
           ("Horses are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'u-food')]
    }),

    ("Do they bite?", {
        'entities': []
    }),

    ("horses are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'U-food')]
    }),

    ("horses pretend to care about your feelings", {
        'entities': [(0, 6, 'U-food')]
    }),

    ("they pretend to care about your feelings, those horses", {
        'entities': [(48, 54, 'U-food')]
    }),

    ("horses?", {
        'entities': [(0, 6, 'U-food')]
    })]

In [34]:
LABEL = 'FOOD'

In [4]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy


# new entity label
LABEL = 'ANIMAL'

# training data
# Note: If you're using an existing model, make sure to mix in examples of
# other entity types that spaCy correctly recognized before. Otherwise, your
# model might learn the new type, but "forget" what it previously knew.
# https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting
TRAIN_DATA = [
    ("Horses are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("Do they bite?", {
        'entities': []
    }),

    ("horses are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("horses pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("they pretend to care about your feelings, those horses", {
        'entities': [(48, 54, 'ANIMAL')]
    }),

    ("horses?", {
        'entities': [(0, 6, 'ANIMAL')]
    }),
    
]




In [5]:

@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    new_model_name=("New model name for model meta.", "option", "nm", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int))
def new_entity_train(model=None, new_model_name='animal', output_dir=None, n_iter=20):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe('ner')

    ner.add_label(LABEL)   # add new entity label to entity recognizer
    if model is None:
        optimizer = nlp.begin_training()
    else:
        # Note that 'begin_training' initializes the models, so it'll zero out
        # existing entity types.
        optimizer = nlp.entity.create_optimizer()



    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update([text], [annotations], sgd=optimizer, drop=0.35,
                           losses=losses)
            print(losses)

    # test the trained model
    test_text = 'horses pretend to care about your feelings.'
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)




In [6]:
new_entity_train()

Created blank 'en' model
{'ner': 36.59821707998901}
{'ner': 15.447522977300416}
{'ner': 3.29271656725482}
{'ner': 7.724493866272157}
{'ner': 1.4510453513781678e-06}
{'ner': 3.230128762872222e-07}
{'ner': 1.990365256912999}
{'ner': 1.019094760735384}
{'ner': 0.007425690924430042}
{'ner': 3.73022774899622e-09}
{'ner': 2.182485714584857}
{'ner': 2.71651157534157e-07}
{'ner': 1.5312407467885668e-06}
{'ner': 2.1816067941402232e-14}
{'ner': 0.04199000131922784}
{'ner': 2.5476107215130123e-05}
{'ner': 2.244229255401748e-12}
{'ner': 3.805022853800884e-08}
{'ner': 9.608647034049874e-14}
{'ner': 0.07771080743624467}
Entities in 'horses pretend to care about your feelings.'
ANIMAL horses


In [21]:
import prodigy

## Rule-based match test

In [7]:
import spacy
from spacy.matcher import Matcher

In [29]:
nlp = spacy.load('en_core_web_md')
matcher = Matcher(nlp.vocab)


In [31]:
pattern = [ {'SHAPE': 'dddd'}]
matcher.add('KEYCARD_NUMBER', None, pattern)

doc = nlp(u"789345")
print([t.text for t in doc])
matches = matcher(doc)
for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

['789345']
789345
789345


doc = nlp("This is an IP address: 7893452")
doc[6].shape_ # find out the 'real shape' of the token that matters